# 五、信息完整性
## 讯息鉴别码（Message Authentication Code）
发送者对信息通过公钥使用MAC签名算法获得标签，将信息和签名标签发送给接收者。接收者对信息通过公钥使用MAC认证算法，看发送获得的标签与算法生成的标签是否>匹配。返回匹配或不匹配的布尔逻辑符。

MAC必须使用共享密钥，不使用密钥类似循环冗余校验（CRC）检查传输错误的方法，只能检查随机错误，不能应对恶意攻击。
对于MAC，攻击者的最终目标是存在伪造（existential forgery），也即复制一对正确的信息和标签。攻击者可以采用选择信息攻击（chosen message attack）。

一个安全的MAC：1）攻击者对新信息无法生成正确的标签。2）给定一组信息和标签，攻击者无法对信息生成新标签并通过验证。
MAC可以保证在病毒入侵之后，检测出所有改动过的文件。但无法应对认证文件路径交换的问题。

## 基于伪随机函数（PRF）的讯息鉴别码（MAC）
任何一个安全的PRF，只要其生成的伪随机数位数足够长（大于64位），都是安全的MAC。一个推论是，对安全的PRF，截取其生成的伪随机数作为MAC的标签，只要截取之>后的长度足够长，这个MAC依然是安全的。

对于PRF来说，主要问题是如AES这样的分组加密方式，仅适用于小文件，如何转化使其适用于大文件。

目前有两大类构造方法，一类是CBC-MAC，广泛应用于银行系统。一类是HMAC，应用于网络传输，SSL、IPsec、SSH均采用了这类方法。

## CBC-MAC和NMAC
CBC-MAC：对第一个模块用密钥一使用PRF加密，得到的密文与下一个模块异或之后作为新一轮的输入，继续用密钥一和PRF加密。最后一轮产生的密文，用密钥二使用相同
的PRF加密。

CBC-MAC不使用最后一轮新密钥加密的破解方法：先使用单个模块长度的m，得到标签t=F（k, m）。则对于双模块长度的明文m||m⊕t，其标签依然是t。

NMAC：PRF产生于密钥同长的密文，对第一个模块使用密钥一加密，得到的结果直接作为第二个模块加密的密钥。最后一轮得到的密文（与密钥同长），补齐成为模块长度
后，使用密钥二，用相同的PRF加密。

NMAC不使用最后一轮新密钥加密的破解方法：先用m加密得到t，之后对任意单个模块长度的w，用m||w作为信息输入，新的密文是F（t, m||w）。

如果可以获得CBC-MAC的共|X|<sup>1⁄2</sup>组信息，NMAC的共|Y|<sup>1⁄2</sup>组信息，则相应方法会变得不安全。因为对两组加密方法来说，均存在不同信息x和y，
但加密标签相同的情况。这种情况下，很明显x||w和y||w的加密标签也会是相同的。

## MAC补齐
使用全0补齐是不安全的。

ISO标准：不对齐的原文，使用一个1和若干个0补齐。对齐的原文，添加首位为1其余为0的虚拟模块。

NIST标准：在ISO标准基础上，不使用虚拟模块。而是对最后一位区分对待:不对齐补齐后的文本，与密钥一（记之前的密钥为密钥零）异或后作为输入；对齐的文本，与>密钥二异或后作为输入。

## PMAC和Carter-Wegman MAC
ECBC和NMAC都是序列生成标签的，而PMAC是一种并行式的标签生成方法。

PMAC：用一个简单的函数，包括密钥一和序列号作为输入，函数输出与序号对应的模块做异或运算，然后进入伪随机函数用密钥二加密。各模块最终伪随机函数加密的结>果一起做异或运算，在用相同的伪随机函数使用密钥二加密得到最终的标签。

PMAC具有增量特征，仅改变原文其中要给模块，可以迅速地计算出对应的标签。

以上介绍的所有讯息鉴别码，均基于伪随机函数。而Carter-Wegman MAC是基于一次性讯息鉴别码，类似于一次性密码本的原理，速度更快。


# 六、抗碰撞
## 简介
哈希函数将信息转化为长度相比而言极小的标签。碰撞是指，对于两条不同的信息，其哈希值相同。理论上来说，根据鸽巢定律，碰撞在哈希过程中是一定存在的。抗碰>撞是指，给定信息和对应的哈希值，无法通过有效的算法找到哈希值相同的其它信息。

SHA-256就是典型的抗碰撞哈希函数。

有了抗碰撞哈希函数，讯息鉴别码过程中，签名函数和认证函数均使用哈希之后的特征值作为信息输入。可以证明，只要哈希函数抗碰撞，MAC就是安全的。

Linux各发行版的软件，就是通过在公有空间以只读形式存储所有软件包的哈希值，用户下载软件之后可以认证下载内容是否正确。

## 利用生日悖论的攻击
假设哈希值为n位，则哈希值的取值空间是2<sup>n</sup>。那么只需要取2<sup>n/2</sup>组信息，就有大约50%的概率可以找到至少一对不同的信息具有相同的哈希值。

这个思路类似于加密算法攻击中的穷举搜索。

## Merkle-Damgard范式
只要能够构建一个对单一模块抗碰撞的哈希函数，就可以实现多模块的哈希函数。

将信息分割为多个模块，最后一个模块用“1000……||信息长度”来补齐。对第一个模块，用固定的初始向量（IV）和模块一作为输入信息，使用压缩函数，生成的结果作为>下一个模块的输入向量。

可以证明，这样的构造方法得到的多模块哈希函数是抗碰撞的。

## HMAC
直接使用哈希函数作为MAC是不安全的。实际中使用的是：
S(k,m)=H(k⊕opad||H(k⊕ipad||m))

## 时间攻击
以Python为例，==运算逐字节进行对比，找到第一个字节左右变量不想等，即返回结果。如果MAC的认证算法使用的是这样的计算，则不同的错误标签，返回错误的计算时
间也不相同。攻击者可以据此逐位猜测标签。

方案一：逐字节对比出结果后，再一起做或运算。则对于所有标签，计算时间均相同。

方案二：将提供的标签和认证算法计算出的标签哈希之后再做==对比。
